In [133]:
#imports
import pysat
import numpy as np
import matplotlib as plt
import pandas as pd
from pysat.solvers import Glucose3
from pysat.card import *
import time
import datetime

In [134]:
q=6
#dictionaryt kell csinálni
valtozok=dict()
k=1
for i in range(q**2+q+1):
    for j in range(q**2+q+1):
        valtozok[tuple([i, j])]= k
        k=k+1

In [135]:
ssolver = Glucose3()
normalforma=CNF()

In [136]:
# maradok annál, hogy külön sor- és oszlopfeltételek legyenek, mert kipróbáltam azt is, hogy az egész mátrixban (q+1)*(q^2+q+1)
# legyen, és ahhoz 882575 db változót használt.

#minden sorban pontosan q+1 db 1-es legyen
st1 = datetime.datetime.now()
maxvaltozo=(q**2+q+1)**2
for i in range(q**2+q+1):
    l1=[]
    for j in range(q**2+q+1):
        l1=l1+[valtozok[(i,j)]]
        
    cnf=CardEnc.equals(lits=l1, bound=q+1, top_id=maxvaltozo)
    for cl in cnf.clauses:
        normalforma.append(cl)
    
    #az abselemek lista tartalmazza az eddig használt változók abszolút értékét
    #a top_id frissítéséhez használom a maxvaltozo-t
    abselemek=[]
    for kloz in normalforma.clauses:
        l2=[abs(elem) for elem in kloz]
        abselemek=abselemek+l2
    maxvaltozo=max(abselemek)
#print(normalforma.clauses)
et1 = datetime.datetime.now()
blokkido1=et1-st1
print(blokkido1)

0:02:16.003921


In [137]:
#minden oszlopban pontosan q+1 db 1-es legyen
st2 = datetime.datetime.now()
for i in range(q**2+q+1):
    l1=[]
    for j in range(q**2+q+1):
        l1=l1+[valtozok[(j,i)]]
        
    cnf=CardEnc.equals(lits=l1, bound=q+1, top_id=maxvaltozo)
    for cl in cnf.clauses:
        normalforma.append(cl)
    
    #az abselemek lista tartalmazza az eddig használt változók abszolút értékét
    #a top_id frissítéséhez használom a maxvaltozo-t
    abselemek=[]
    for kloz in normalforma.clauses:
        l2=[abs(elem) for elem in kloz]
        abselemek=abselemek+l2
    maxvaltozo=max(abselemek)
    
et2 = datetime.datetime.now()
blokkido2=et2-st2
print(blokkido2)

0:33:44.581220


In [138]:
#már csak az kell, hogy ne legyen csupa 1-es négyzet, vagyis i, k sor és j, l oszlop, hogy ij, il, kj, és kl is igaz
for i in range(q**2+q+1):
    for j in range(q**2+q+1):
        for k in range(q**2+q+1):
            for l in range(q**2+q+1):
                x1=valtozok[(i,j)]
                x2=valtozok[(i,l)]
                x3=valtozok[(k,j)]
                x4=valtozok[(k,l)]
                if (x1!=x2 and x1!=x3 and x1!=x4 and x2!=x3 and x2!=x4 and x3!=x4):
                    normalforma.clauses.append([-x1, -x2, -x3, -x4])

In [139]:
print(maxvaltozo)

45193


In [140]:
#beleírom a plusz feltevéseket, amiket a megoldások szimmetriája miatt feltehetünk
#az első sor első q+1 eleme 1, a többi 0
for i in range(q+1):
    normalforma.clauses.append([valtozok[0, i]])
for i in range(q+1, q**2+q+1, 1):
    normalforma.clauses.append([-valtozok[0, i]])
    
#az első oszlop 2. elemétől a q+1-edik oszlop utolsó eleméig felírom a feltételeket, i lesz az oszlopindex, j a sorindex
for i in range(q+1):
    for j in range(1, q**2+q+1, 1):
        if (j >= i*q+1 and j<(i+1)*q+1):
            normalforma.clauses.append([valtozok[j,i]])
        else:
            normalforma.clauses.append([-valtozok[j, i]])

#a q+2-edik oszlop formáját is elő lehet írni
for i in range(1, q**2+q+1, 1):
    if i%q==1:
        normalforma.clauses.append([valtozok[i, q+1]])
    else:
        normalforma.clauses.append([-valtozok[i, q+1]])

In [141]:
#print(normalforma.clauses)

In [ ]:
#beadom a sat-solvernek a normálformát
st3=datetime.datetime.now()
for kloz in normalforma.clauses:
    ssolver.add_clause(kloz)
    
# megoldás kiszámítása
ssolver.solve()
ans=ssolver.get_model()
print(ans)
et3=datetime.datetime.now()
blokkido3=et3-st3
print(blokkido3)

In [131]:
#csinálok egy listák listáját a megoldásból, amiben csak a nekünk releváns változók szerepelnek és csak 1-0 elemek vannak

megoldaslista=[]
for i in range(0, (q**2+q+1)**2, q**2+q+1):
    l1=ans[i:i+q**2+q+1]
    l2=[]
    for elem in l1:
        if elem>0:
            l2=l2+[1]
        else:
            l2=l2+[0]
    megoldaslista.append(l2)
        
#kiprintelem a megoldást mátrix alakban

szepmatrix=np.array(megoldaslista)
print(szepmatrix)

[[1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 1 0 0 0 0 0 0 1 1 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 1 0 0 1 0 1 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 1 1 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 1 1 0 1 0 0]
 [1 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 1 0]
 [0 1 0 0 0 0 1 0 1 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 0 0 1 1 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 1 0]
 [0 1 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 1 0 0]
 [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 1 0 0 0 1 0 0 1]
 [0 1 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 1 0 0 0 1 0 0 1 0 0 0 0 0 0]
 [0 0 1 0 0 0 1 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1]
 [0 0 1 0 0 0 0 0 0 0 1 1 0 0 1 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0]
 [0 0 1 0 0 0 0 0 1 0 0 0 0 1 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 1 1 0]
 [0 0 1 0 0 0 0 0 0 0 0 0

In [132]:
print(blokkido1+blokkido2+blokkido3)

0:03:02.371362
